In [23]:
!pip install datasets transformers[sentencepiece]
!pip install faiss-gpu

import time
import torch
from transformers import AutoTokenizer, AutoModel
import pickle
from os.path import exists
from datasets import Dataset, load_dataset

def load_embeddings():
    filename = "https://drive.google.com/file/d/1mQswT2SyXJbtvVlfwwGVAdQNEquwqUXU/"
    infile = open(filename,'rb')
    my_dataset = pickle.load(infile)
    infile.close()
    return my_dataset

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    # cls_pooling returns embedding of last output
    return model_output.last_hidden_state[:, 0]

# load pretrained tokenizer and model
model_name = "allenai-specter"
MODEL_PREFIX = "sentence-transformers/"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PREFIX + model_name)
model = AutoModel.from_pretrained(MODEL_PREFIX + model_name)
device = torch.device("cuda")
model.to(device)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31116, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [15]:
!ls /content/

embeddings.pkl	sample_data


In [24]:
import gdown
import pickle

def load_embeddings():
    # filename = "/content/sample_data/california_housing_train.csv"
    filename = "/content/embeddings.pkl"
    # url = "https://drive.google.com/file/d/1mQswT2SyXJbtvVlfwwGVAdQNEquwqUXU/view?usp=sharing"
    # gdown.download(url=url, output=filename, quiet=False)
    f = open(filename,'rb')
    my_dataset = pickle.load(f)
    f.close()
    return my_dataset

embeddings_dataset = load_embeddings()
#print(embeddings_dataset)

FileNotFoundError: ignored

In [79]:
#@title Semantic Search {display-mode: "form"}
query = "openai" #@param {type: "string"}

In [80]:
import pandas as pd

start_time = time.time()
question_embedding = get_embeddings([query]).cpu().detach().numpy()

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

end_time = time.time()
print(f"Elapsed time: {int((end_time - start_time)*1000)}ms")

for _, row in samples_df.iterrows():
    print(f"({row.scores:.2f})\t{row.title}\t{row.url}")

Elapsed time: 1103ms
(160.31)	OpenAI Gym	http://arxiv.org/abs/1606.01540v1
(170.19)	The AI Index 2021 Annual Report	http://arxiv.org/abs/2103.06312v1
(186.65)	Adam: A Method for Stochastic Optimization	http://arxiv.org/abs/1412.6980v9
(191.23)	The 30-Year Cycle In The AI Debate	http://arxiv.org/abs/1810.04053v1
(194.40)	The Benchmark Lottery	http://arxiv.org/abs/2107.07002v1
(195.87)	Science Facing Interoperability as a Necessary Condition of Success and Evil	http://arxiv.org/abs/2202.02540v1
(200.45)	Learning Curve Theory	http://arxiv.org/abs/2102.04074v1
(202.56)	Reconciling modern machine learning practice and the bias-variance trade-off	http://arxiv.org/abs/1812.11118v2
(203.58)	Accounting for the Neglected Dimensions of AI Progress	http://arxiv.org/abs/1806.00610v1
(204.46)	A Berkeley View of Systems Challenges for AI	http://arxiv.org/abs/1712.05855v1
